In [1]:
import sys
sys.path.append("../src")

In [2]:
import pandas as pd

# URLs de los datasets
glass_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data"
iris_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Columnas de los datasets
glass_columns = [
    "Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Type"
]
iris_columns = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Class"]

# Cargar los datasets directamente desde las URLs
glass_df = pd.read_csv(glass_url, names=glass_columns, header=None)
iris_df = pd.read_csv(iris_url, names=iris_columns, header=None)

# Verificar la carga
print("Glass Dataset:")
print(glass_df.head())

print("\nIris Dataset:")
print(iris_df.head())


Glass Dataset:
   Id       RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  Type
0   1  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0     1
1   2  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0     1
2   3  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0     1
3   4  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0     1
4   5  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0     1

Iris Dataset:
   SepalLength  SepalWidth  PetalLength  PetalWidth        Class
0          5.1         3.5          1.4         0.2  Iris-setosa
1          4.9         3.0          1.4         0.2  Iris-setosa
2          4.7         3.2          1.3         0.2  Iris-setosa
3          4.6         3.1          1.5         0.2  Iris-setosa
4          5.0         3.6          1.4         0.2  Iris-setosa


In [4]:
from preprocess import min_max_scaler

# Preprocesamiento para el Glass Dataset
glass_features = glass_df.drop(columns=["Id", "Type"])  # Eliminar columnas no numéricas
glass_target = glass_df["Type"]

# Normalizar con min_max_scaler
glass_features_normalized = min_max_scaler(glass_features.values.tolist())
glass_data_normalized = pd.DataFrame(glass_features_normalized, columns=glass_features.columns)
glass_data_normalized["Type"] = glass_target

# Preprocesamiento para el Iris Dataset
iris_features = iris_df.drop(columns=["Class"])
iris_target = iris_df["Class"]

# Convertir clases a valores numéricos
iris_target_numeric = iris_target.map({
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2
})

# Normalizar con min_max_scaler
iris_features_normalized = min_max_scaler(iris_features.values.tolist())
iris_data_normalized = pd.DataFrame(iris_features_normalized, columns=iris_features.columns)
iris_data_normalized["Class"] = iris_target_numeric

# Verificar el resultado
print("Glass Dataset Preprocesado:")
print(glass_data_normalized.head())

print("\nIris Dataset Preprocesado:")
print(iris_data_normalized.head())


Glass Dataset Preprocesado:
         RI        Na        Mg        Al        Si         K        Ca   Ba  \
0  0.432836  0.437594  1.000000  0.252336  0.351786  0.009662  0.308550  0.0   
1  0.283582  0.475188  0.801782  0.333333  0.521429  0.077295  0.223048  0.0   
2  0.220808  0.421053  0.790646  0.389408  0.567857  0.062802  0.218401  0.0   
3  0.285777  0.372932  0.821826  0.311526  0.500000  0.091787  0.259294  0.0   
4  0.275241  0.381955  0.806236  0.295950  0.583929  0.088567  0.245353  0.0   

    Fe  Type  
0  0.0     1  
1  0.0     1  
2  0.0     1  
3  0.0     1  
4  0.0     1  

Iris Dataset Preprocesado:
   SepalLength  SepalWidth  PetalLength  PetalWidth  Class
0     0.222222    0.625000     0.067797    0.041667      0
1     0.166667    0.416667     0.067797    0.041667      0
2     0.111111    0.500000     0.050847    0.041667      0
3     0.083333    0.458333     0.084746    0.041667      0
4     0.194444    0.666667     0.067797    0.041667      0


In [5]:
from validation import hold_out

# Parámetro r para dividir los datos
test_size = 0.3  # 30% para prueba, 70% para entrenamiento

# Hold-Out para el Glass Dataset
X_glass = glass_data_normalized.drop(columns=["Type"]).values.tolist()
y_glass = glass_data_normalized["Type"].values.tolist()

X_train_glass, X_test_glass, y_train_glass, y_test_glass = hold_out(X_glass, y_glass, test_size=test_size, random_seed=42)

print("Hold-Out Glass Dataset:")
print(f"Tamaño de entrenamiento: {len(X_train_glass)}, Tamaño de prueba: {len(X_test_glass)}")

# Hold-Out para el Iris Dataset
X_iris = iris_data_normalized.drop(columns=["Class"]).values.tolist()
y_iris = iris_data_normalized["Class"].values.tolist()

X_train_iris, X_test_iris, y_train_iris, y_test_iris = hold_out(X_iris, y_iris, test_size=test_size, random_seed=42)

print("\nHold-Out Iris Dataset:")
print(f"Tamaño de entrenamiento: {len(X_train_iris)}, Tamaño de prueba: {len(X_test_iris)}")


Hold-Out Glass Dataset:
Tamaño de entrenamiento: 150, Tamaño de prueba: 64

Hold-Out Iris Dataset:
Tamaño de entrenamiento: 105, Tamaño de prueba: 45


In [6]:
from validation import k_fold_cross_validation

# Parámetro K para dividir los datos
k = 5  # Número de pliegues

# K-Fold Cross-Validation para el Glass Dataset
folds_glass = k_fold_cross_validation(X_glass, y_glass, k=k)

print("K-Fold Glass Dataset:")
for i, (X_train, X_val, y_train, y_val) in enumerate(folds_glass, 1):
    print(f"  Fold {i}:")
    print(f"    Tamaño de entrenamiento: {len(X_train)}, Tamaño de validación: {len(X_val)}")

# K-Fold Cross-Validation para el Iris Dataset
folds_iris = k_fold_cross_validation(X_iris, y_iris, k=k)

print("\nK-Fold Iris Dataset:")
for i, (X_train, X_val, y_train, y_val) in enumerate(folds_iris, 1):
    print(f"  Fold {i}:")
    print(f"    Tamaño de entrenamiento: {len(X_train)}, Tamaño de validación: {len(X_val)}")


K-Fold Glass Dataset:
  Fold 1:
    Tamaño de entrenamiento: 172, Tamaño de validación: 42
  Fold 2:
    Tamaño de entrenamiento: 172, Tamaño de validación: 42
  Fold 3:
    Tamaño de entrenamiento: 172, Tamaño de validación: 42
  Fold 4:
    Tamaño de entrenamiento: 172, Tamaño de validación: 42
  Fold 5:
    Tamaño de entrenamiento: 172, Tamaño de validación: 42

K-Fold Iris Dataset:
  Fold 1:
    Tamaño de entrenamiento: 120, Tamaño de validación: 30
  Fold 2:
    Tamaño de entrenamiento: 120, Tamaño de validación: 30
  Fold 3:
    Tamaño de entrenamiento: 120, Tamaño de validación: 30
  Fold 4:
    Tamaño de entrenamiento: 120, Tamaño de validación: 30
  Fold 5:
    Tamaño de entrenamiento: 120, Tamaño de validación: 30


In [7]:
from validation import leave_one_out

# Leave-One-Out para el Glass Dataset
loo_glass = leave_one_out(X_glass, y_glass)

print("Leave-One-Out Glass Dataset:")
for i, (X_train, X_val, y_train, y_val) in enumerate(loo_glass[:5], 1):  # Mostrar solo las primeras 5 iteraciones
    print(f"  Iteración {i}:")
    print(f"    Tamaño de entrenamiento: {len(X_train)}, Tamaño de validación: {len(X_val)}")

# Leave-One-Out para el Iris Dataset
loo_iris = leave_one_out(X_iris, y_iris)

print("\nLeave-One-Out Iris Dataset:")
for i, (X_train, X_val, y_train, y_val) in enumerate(loo_iris[:5], 1):  # Mostrar solo las primeras 5 iteraciones
    print(f"  Iteración {i}:")
    print(f"    Tamaño de entrenamiento: {len(X_train)}, Tamaño de validación: {len(X_val)}")


Leave-One-Out Glass Dataset:
  Iteración 1:
    Tamaño de entrenamiento: 213, Tamaño de validación: 1
  Iteración 2:
    Tamaño de entrenamiento: 213, Tamaño de validación: 1
  Iteración 3:
    Tamaño de entrenamiento: 213, Tamaño de validación: 1
  Iteración 4:
    Tamaño de entrenamiento: 213, Tamaño de validación: 1
  Iteración 5:
    Tamaño de entrenamiento: 213, Tamaño de validación: 1

Leave-One-Out Iris Dataset:
  Iteración 1:
    Tamaño de entrenamiento: 149, Tamaño de validación: 1
  Iteración 2:
    Tamaño de entrenamiento: 149, Tamaño de validación: 1
  Iteración 3:
    Tamaño de entrenamiento: 149, Tamaño de validación: 1
  Iteración 4:
    Tamaño de entrenamiento: 149, Tamaño de validación: 1
  Iteración 5:
    Tamaño de entrenamiento: 149, Tamaño de validación: 1
